In [59]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
try:
    conexion = psycopg2.connect(
    database="laaboratorio3",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432")
except OperationalError as e:
    if e.pgcode ==errorcodes.INVALID_PASSWORD:
        print("La constraseña es incorrecta")
    elif e.pgcode ==errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")

In [61]:
cursor = conexion.cursor()

In [62]:
conexion.rollback()

# Ejercicio 1. Queries Generales

#### Ejercicio 1.1. Calcula el promedio más bajo y más alto de temperatura

In [63]:
query1 = """
select avg(temperatura), max(temperatura), min(temperatura) from tiempo;
"""

cursor.execute(query1)
df_query1 = pd.DataFrame(cursor.fetchall(),columns=["Media","Max","Min"])
# df_query1.columns = ["Media","Max","Min"]
df_query1


,Media,Max,Min
0,21.2498387720882239,32,9


#### 1.2. Obtén los municipios en los cuales coincidan las medias de la sensación térmica y de la temperatura.

In [64]:
query2 = """
select id_municipio
from tiempo
group by id_municipio
having avg(temperatura) = avg(sensacion_termica);
"""

cursor.execute(query2)
df_query1 = pd.DataFrame(cursor.fetchall(), columns=["Municipios coincidentes"])
df_query1.head(5)

,Municipios coincidentes
0,pradena-del-rincon
1,robregordo
2,pinuecar-gandullas
3,atazar-el
4,collado-mediano


#### 1.3. Obtén el local más cercano de cada municipio

In [65]:
query3 = """
select m."nombre", categoria, min(distance)
from lugar l
inner join "municipios" m on m."id_municipio" = l."id_municipio" 
group by m."nombre", categoria
"""

cursor.execute(query3)
df_query3 = pd.DataFrame(cursor.fetchall(), columns=["Municipios","Local","Distancia"])
df_query3.head()

,Municipios,Local,Distancia
0,Villaviciosa de Odón,Monument,747.0
1,Navacerrada,Park,95.0
2,Tres Cantos,Monument,577.0
3,Meco,Park,925.0
4,Tres Cantos,TV Station,1106.0


#### 1.4. Localiza los municipios que posean algún localizador a una distancia mayor de 2000 y que posean al menos 25 locales.

In [66]:
query4 = """
select m."nombre", categoria, distance
from lugar l
inner join "municipios" m on m."id_municipio" = l."id_municipio" 
group by m."nombre", categoria, l.distance, address
having distance > 2000
"""

cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(), columns=["Municipios","Local","Distancia"])
df_query4.head(5)

,Municipios,Local,Distancia
0,Madrid,Film Studio,2002.0


#### 1.5. Teniendo en cuenta que el viento se considera leve con una velocidad media de entre 6 y 20 km/h, moderado con una media de entre 21 y 40 km/h, fuerte con media de entre 41 y 70 km/h y muy fuerte entre 71 y 120 km/h. Calcula cuántas rachas de cada tipo tenemos en cada uno de los días. Este ejercicio debes solucionarlo con la sentencia CASE de SQL (no la hemos visto en clase, por lo que tendrás que buscar la documentación).

In [67]:
query5 = """
select fecha, id_municipio, velocidad_viento,
case 
    when velocidad_viento <= 6 AND velocidad_viento <= 20 THEN 'leve'
    when velocidad_viento <= 21 AND velocidad_viento <= 40 THEN 'moderado'
    when velocidad_viento <= 41 AND velocidad_viento <= 70 THEN 'fuerte'
    when velocidad_viento <= 71 AND velocidad_viento <= 120 THEN 'muy fuerte'
END
from tiempo t 
"""
cursor.execute(query5)
df_query5 = pd.DataFrame(cursor.fetchall(), columns = ["Fecha","id_municipio","Velocidad_viento(km/h)","tipo_viento"])
df_query5.sample(10)

,Fecha,id_municipio,Velocidad_viento(km/h),tipo_viento
7317,2024-08-29,villamanta,7,moderado
6043,2024-08-29,tielmes,22,fuerte
5033,2024-08-29,reduena,5,leve
3425,2024-08-30,majadahonda,15,moderado
4302,2024-08-30,paracuellos-de-jarama,12,moderado
3134,2024-08-29,humanes-de-madrid,8,moderado
7049,2024-08-29,vellon-el,5,leve
1847,2024-08-31,cobena,11,moderado
6161,2024-08-31,torrejon-de-ardoz,9,moderado
7099,2024-08-29,venturada,9,moderado


In [86]:
conexion.rollback()

# Ejercicio2. Vistas

#### 2.1. Crea una vista que muestre la información de los locales que tengan incluido el código postal en su dirección.

In [69]:
# La dejo para más tarde, no soy capaz de comprender las cosas
query6 = """
create view vista3 as
select id_municipio
from lugar l
where address ~ '/d{5}'
"""
cursor.execute(query6)

#### 2.2. Crea una vista con los locales que tienen más de una categoría asociada.

In [70]:
query6 = """
create view Tabla_categorias as
select name,count(categoria) over (partition by categoria) as "cat" 
from lugar l ;
"""
cursor.execute(query6)

#### 2.3 Crea una vista que muestre el municipio con la temperatura más alta de cada día

In [71]:
query6 = """
create view muni_temp as
select id_municipio, max(temperatura) over (partition by fecha) 
from tiempo t ;
"""
cursor.execute(query6)

#### 2.4. Crea una vista con los municipios en los que haya una probabilidad de precipitación mayor del 100% durante mínimo 7 horas.

In [72]:
query6 = """
create view prob100 as
select id_municipio
from tiempo t
where prob_precip = '100' OR prob_precip = 'riesgo'
"""
cursor.execute(query6)

#### 2.5. Obtén una lista con los parques de los municipios que tengan algún castillo.

In [73]:
query6 = """
create view castillo1 as
select name
from lugar l
where categoria = 'Castle'
"""
cursor.execute(query6)
# He tenido que ejecutar la obtencion en DBBeaver, esta la obtención en la foto: fotinchi.png

In [83]:
conexion.rollback()

# Ejercicio 3. Tablas Temporales

#### 3.1. Crea una tabla temporal que muestre cuántos días han pasado desde que se obtuvo la información de la tabla AEMET.

In [75]:
query6 = """
CREATE TEMPORARY TABLE Dias_Desde_Ultima_Fecha AS
SELECT 
    id_tiempo, 
    id_municipio, 
    fecha, 
    (CURRENT_DATE - fecha) AS dias_transcurridos
FROM 
    tiempo;
"""
cursor.execute(query6)

#### 3.2. Crea una tabla temporal que muestre los locales que tienen más de una categoría asociada e indica el conteo de las mismas

In [76]:
query6 = """
CREATE TEMPORARY TABLE Locales_Multiples_Categorias AS
SELECT 
    id_lugar, 
    COUNT(DISTINCT categoria) AS total_categorias
FROM lugar
GROUP BY id_lugar
HAVING COUNT(DISTINCT categoria) > 1;
"""
cursor.execute(query6)

#### 3.3. Crea una tabla temporal que muestre los tipos de cielo para los cuales la probabilidad de precipitación mínima de los promedios de cada día es 5.

#### 3.4. Crea una tabla temporal que muestre el tipo de cielo más y menos repetido por municipio.

In [82]:
query6 = """
CREATE TEMPORARY TABLE Cielo_Mas_Menos_Repetido AS
WITH Cielo_Frecuencias AS (
    SELECT 
        id_municipio, 
        cielo, 
        COUNT(*) AS frecuencia
    FROM 
        tiempo
    GROUP BY 
        id_municipio, cielo
)
SELECT 
    municipios.nombre AS municipio,
    (SELECT cielo FROM Cielo_Frecuencias AS cf1 WHERE cf1.id_municipio = cf.id_municipio ORDER BY frecuencia DESC LIMIT 1) AS cielo_mas_repetido,
    (SELECT cielo FROM Cielo_Frecuencias AS cf2 WHERE cf2.id_municipio = cf.id_municipio ORDER BY frecuencia ASC LIMIT 1) AS cielo_menos_repetido
FROM 
    Cielo_Frecuencias AS cf
JOIN 
    municipios ON municipios.id_municipio = cf.id_municipio
GROUP BY 
    municipios.nombre, cf.id_municipio;

"""
cursor.execute(query6)

# Ejercicio 4. SUBQUERIES

#### 4.1. Necesitamos comprobar si hay algún municipio en el cual no tenga ningún local registrado.

In [96]:
query4 = """
SELECT nombre 
FROM municipios
WHERE id_municipio NOT IN (
    SELECT id_municipio 
    FROM lugar
);
"""

cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(), columns = ["Sin Local"])
df_query4.head(5)

,Sin Local
0,La Acebeda
1,Alameda del Valle
2,Aldea del Fresno
3,Ambite
4,Anchuelo


#### 4.2. Averigua si hay alguna fecha en la que el cielo se encuente "Muy nuboso con tormenta".

In [88]:
query4 = """
SELECT fecha 
FROM tiempo
WHERE cielo = 'Muy nuboso con tormenta'; 
"""
# Creo que no jhace falta hacer una subquery, 
cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(),columns=["Fecha"])
df_query4.head(5)

,Fecha
0,2024-08-29
1,2024-08-29
2,2024-08-29


#### 4.3. Encuentra los días en los que los avisos sean diferentes a "Sin riesgo".

In [89]:
query4 = """
SELECT fecha 
FROM tiempo
WHERE avisos <> 'Sin riesgo';
"""
# Creo que no jhace falta hacer una subquery, 
cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(),columns=["Fecha"])
df_query4.head(5)

,Fecha
0,2024-08-29
1,2024-08-29
2,2024-08-29
3,2024-08-29
4,2024-08-29


#### 4.4. Selecciona el municipio con mayor número de locales.

In [91]:
query4 = """
SELECT nombre 
FROM municipios
WHERE id_municipio = (
    SELECT id_municipio 
    FROM lugar
    GROUP BY id_municipio
    ORDER BY COUNT(*) DESC
    LIMIT 1
);

"""
cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(),columns=["Localisdad con mas lugares"])
df_query4.head(5)

,Localisdad con mas lugares
0,Madrid


#### 4.5. Obtén los municipios muya media de sensación térmica sea mayor que la media total.

In [92]:
query4 = """
SELECT nombre 
FROM municipios
WHERE id_municipio IN (
    SELECT id_municipio 
    FROM tiempo
    GROUP BY id_municipio
    HAVING AVG(sensacion_termica) > (
        SELECT AVG(sensacion_termica) 
        FROM tiempo
    )
);
"""
cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(),columns=["Sensación Media"])
df_query4.head(5)

,Sensación Media
0,Ajalvir
1,El Álamo
2,Alcalá de Henares
3,Alcobendas
4,Alcorcón


#### 4.6. Selecciona los municipios con más de dos fuentes.

In [93]:
query4 = """
SELECT nombre 
FROM municipios
WHERE id_municipio IN (
    SELECT id_municipio 
    FROM lugar
    WHERE categoria = 'Fountain'
    GROUP BY id_municipio
    HAVING COUNT(*) > 2
);
"""
cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(),columns=["localidad con más de dos fuentes"])
df_query4.head(5)

,localidad con más de dos fuentes
0,Madrid


#### 4.7. Localiza la dirección de todos los estudios de cine que estén abiertod en el municipio de "Madrid".

""


#### 4.8. Encuentra la máxima temperatura para cada tipo de cielo.

In [94]:
query4 = """
SELECT cielo, MAX(temperatura)
FROM tiempo
GROUP BY cielo;
"""
cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(), columns=["Cielo","Maxima temp"])
df_query4.head(5)

,Cielo,Maxima temp
0,Cubierto con lluvia,30
1,Nuboso con lluvia,16
2,Poco nuboso,32
3,Nuboso con lluvia escasa,30
4,Intervalos nubosos con lluvia escasa,31


#### 4.9. Muestra el número de locales por categoría que muy probablemente se encuentren abiertos.

In [95]:
query4 = """
SELECT categoria, COUNT(*) AS locales_abiertos
FROM lugar
WHERE closed_bucket = 'VeryLikelyOpen'
GROUP BY categoria;
"""
# no hace falta subconsulta, creo
cursor.execute(query4)
df_query4 = pd.DataFrame(cursor.fetchall(), columns= ["Lugar","Nº Espacios abiertos"])
df_query4.head(5)

,Lugar,Nº Espacios abiertos
0,Park,199
1,Fountain,3
2,Castle,3
3,Palace,1
4,Bridge,2
